# Compute PSD, SNR, and Z-Score of Pilot Data

## Goals:
1. **Data Import**
    - Import the preprocessed data from npz and json files

2. **Feature Extraction**
    - Calculate the Power Spectral Density (PSD) for each epoch.
    - Calculate the Signal-to-Noise Ratio (SNR) for each epoch (for use in stats)
    - Calculate the Z-score for each epoch (used to determine signal cutoff):
        * **Formula**:   PSD Z-score = (PSD(single trial) - Mean PSD of baseline trials) / Std PSD of baseline trials
        
3. **Data Formatting**
    - Save the data and export to an excel sheet
        * Get the average of all epochs for each unique stimulus

# Import Libraries

In [1]:
# Standard libraries
import json
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt

# Custom libraries
from Functions import processing

# Import Epoched Data and Settings

In [8]:
files = [  
    "sub-P001_ses-S001_task-T1_run-001_eeg"   
]

subject_ids = [file.split('_')[0] for file in files]
unique_subject_ids = list(set(subject_ids))

# Preallocate variables
eeg_epochs = [None] * len(files)
settings = [None] * len(files)

# Import data
for f, file in enumerate(files):
    # Import EEG data, since it is stored in a compressed numpy file (.npz) we need to use the np.load function 
    loaded_data = np.load(f"Data\\Masters_testing\\{file}.npz", allow_pickle=True)

    # Access the data for each stimulus
    eeg_epochs[f] = {stim_label: loaded_data[stim_label] for stim_label in loaded_data.files}

    # Import settings
    with open(f"Data\\Masters_testing\\{file}.json", "r") as file_object:
        settings[f] = json.load(file_object)

# Compute PSD for all Epochs

Make sure to include baseline epochs for later calculation of Z-score
Compute PSD for every epoch (and average for repeated labels after)

# Compute SNR for all Epochs


# Compute Z-score for all Epochs

# Save Data